If you haven't got the influxdb and grafana containers already running the run the following command that will start them up.

In [21]:
!make -f ../Makefile run INFLUXDB_DB=gpudata INFLUXDB_USER=admin INFLUXDB_USER_PASSWORD=password GF_ADMIN_PASSWORD=secret

docker run -d -p 8083:8083 -p 8086:8086 \
-e INFLUXDB_DB=gpudata \
-e INFLUXDB_USER=admin \
-e INFLUXDB_USER_PASSWORD=password \
-v /home/mat/repos/influx_grafana/examples:/var/lib/influxdb \
-e INFLUXDB_ADMIN_ENABLED=true \
--name influxdb \
influxdb
echo "Influx DB gpudata set up waiting on port 8083"
f0a05b672e17af7ccd525c2a3eecef3ebac6fb89fbaab9b971a3a15a3b574a94
Influx DB gpudata set up waiting on port 8083
# create /var/lib/grafana as persistent volume storage
docker run -d -v /var/lib/grafana --name grafana-storage busybox:latest
# start grafana
docker run \
  -d \
  -p 3000:3000 \
  --name=grafana \
  --volumes-from grafana-storage \
  --link influxdb:influxdb \
  -e GF_SECURITY_ADMIN_PASSWORD=secret \
  grafana/grafana
echo "Grafana set up and waiting on port 3000"
1d400ccc1219f11dac025300085d155c1c2d756240ea5524d78d41ba014a7d11
066fb7872cfed8a283d5ae560fcea1e1a15bc63466eaa2e4be5cb2a921711339
Grafana set up and waiting on port 3000
InfluxDB and Grafana started
Influx DB gpudat

In [22]:
import sys

We will install the gpu monitoring utility gpumon

In [2]:
!git clone https://github.com/msalvaris/gpu_monitor.git

Cloning into 'gpu_monitor'...
remote: Counting objects: 174, done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 174 (delta 9), reused 14 (delta 6), pack-reused 157
Receiving objects: 100% (174/174), 106.16 KiB | 0 bytes/s, done.
Resolving deltas: 100% (93/93), done.
Checking connectivity... done.


In [9]:
!$sys.executable -m pip install -e /home/mat/repos/influx_grafana/examples/gpu_monitor

Obtaining file:///home/mat/repos/influx_grafana/examples/gpu_monitor
  Running setup.py develop for gpumon
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
from influxdb import InfluxDBClient, DataFrameClient
from time import sleep
import pandas
from itertools import filterfalse
from toolz import merge,curry, compose, pipe
from gpumon import create_influxdb_writer, nvidia_run_dmon_poll, db_log_context
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here let's simply run the nvidia dmon and use the print statement to pipe it to stdout. It won't stop by itself so you will have to stop it yourself using the stop kernel button in the Jupyter notebook

In [24]:
nvidia_run_dmon_poll(print)

b'#Date       Time        gpu   pwr  temp    sm   mem   enc   dec  mclk  pclk\n'
b'#YYYYMMDD   HH:MM:SS    Idx     W     C     %     %     %     %   MHz   MHz\n'
b' 20171212   12:38:29      0    33    29     0     0     0     0   324   324\n'
b' 20171212   12:38:30      0    33    29     0     0     0     0   324   324\n'
b' 20171212   12:38:31      0    33    28     0     0     0     0   324   324\n'
b' 20171212   12:38:32      0    33    28     0     0     0     0   324   324\n'


KeyboardInterrupt: 

Create the client and connect ot database

In [25]:
client = InfluxDBClient('localhost', 8086, 'admin', 'password', 'gpudata')

In [26]:
client.get_list_database()

[{'name': 'gpudata'}]

Now let's create our influx db writer and pass it into our nvidia deamon. Again it won't automatically stop so stop it after 5-6 seconds.

In [27]:
to_db = create_influxdb_writer(client, series_name="gpu", machine='this_machine')
nvidia_run_dmon_poll(to_db)

KeyboardInterrupt: 

Using the command below we can see what is in the database

In [28]:
client.query('select * from gpu')

ResultSet({'('gpu', None)': [{'time': '2017-12-12T12:38:38Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'time': '2017-12-12T12:38:39Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'time': '2017-12-12T12:38:40Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'time': '2017-12-12T12:38:41Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'time': '2017-12-12T12:38:42Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'time': '2017-12-12T12:38:43Z', 'dec': 0, 'enc': 0, 'gpu': '0', 'machine': 'this_machine', 'mclk': 324, 'mem': 0, 'pclk': 324, 'pwr': 33, 'sm': 0, 'temp': 28}, {'tim

The results above are fine but the formating isn't great. Using the DataFrameClient we can get the results back as a dataframe.

In [29]:
df_client = DataFrameClient('localhost', 8086, 'admin', 'password', 'gpudata')

In [30]:
df_client.query("select * from gpu;")['gpu']

,dec,enc,gpu,machine,mclk,mem,pclk,pwr,sm,temp
2017-12-12 12:38:38+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:39+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:40+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:41+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:42+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:43+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:44+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:45+00:00,0,0,0,this_machine,324,0,324,33,0,28


If you are going to use the gpu monitor in a notebook then your best choice is the logging context below that will setup the connection and keep logging while code is being executed within the context.

In [31]:
with db_log_context('localhost', '8086', 'admin', 'password', 'gpudata', 'gpu', gpu=2, machine='machine2'):
    sleep(5)

INFO:gpumon.influxdb_gpu_logger:Logging GPU to Database localhost
INFO:gpumon.influxdb_gpu_logger:['influxdb_gpu_logger.py', 'localhost', '8086', 'admin', 'password', 'gpudata', 'gpu', '--gpu=2', '--machine=machine2']


In [32]:
df_client.query("select * from gpu;")['gpu']

,dec,enc,gpu,machine,mclk,mem,pclk,pwr,sm,temp
2017-12-12 12:38:38+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:39+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:40+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:41+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:42+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:43+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:44+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:45+00:00,0,0,0,this_machine,324,0,324,33,0,28
2017-12-12 12:38:59+00:00,0,0,2,machine2,324,0,324,33,0,28
2017-12-12 12:39:00+00:00,0,0,2,machine2,324,0,324,33,0,28


Tear everything down and delete the database

In [33]:
!sudo make -f ../Makefile clean

docker stop influxdb grafana-storage grafana
influxdb
grafana-storage
grafana
rm -r data meta wal
docker rm -v influxdb grafana-storage grafana
influxdb
grafana-storage
grafana
